In [1]:
!pip install catboost
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime as dt
from pycaret.regression import *

# 전처리

In [2]:
# 데이터 로드

train = pd.read_csv('데이콘/train.csv')
test = pd.read_csv('데이콘/test.csv')

train[['현본사소속재택근무자수', '중식계', '석식계']] = train[['현본사소속재택근무자수', '중식계', '석식계']].astype('int')
test['현본사소속재택근무자수'] = test['현본사소속재택근무자수'].astype('int')

train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

# 추가 변수 생성

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.week
train['요일'] = train['일자'].dt.weekday
train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
train['식사가능자수'] = train['식사가능자수'].astype('int')

test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.week
test['요일'] = test['일자'].dt.weekday
test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']
test['식사가능자수'] = test['식사가능자수'].astype('int')

train['지표'] = train['석식메뉴'].apply(lambda x : len(x.split()))
train['메뉴x'] = 0
test['메뉴x'] = 0

train.loc[(train['지표'] < 5),'메뉴x']= 1
train = train.drop(['지표'], axis = 1)

lunch_day_mean = train.loc[train['메뉴x'] == 0].groupby(['요일'])[['중식계']].mean()
dinner_day_mean = train.loc[train['메뉴x'] == 0].groupby(['요일'])[['석식계']].mean()
lunch_mon_mean = train.loc[train['메뉴x'] == 0].groupby(['월'])[['중식계']].mean()
dinner_mon_mean = train.loc[train['메뉴x'] == 0].groupby(['월'])[['석식계']].mean()

train['요일중식평균'] = train['요일'].apply(lambda x : lunch_day_mean.loc[x, '중식계'])
train['요일석식평균'] = train['요일'].apply(lambda x : dinner_day_mean.loc[x, '석식계'])
train['월중식평균'] = train['월'].apply(lambda x : lunch_mon_mean.loc[x, '중식계'])
train['월석식평균'] = train['월'].apply(lambda x : dinner_mon_mean.loc[x, '석식계'])
test['요일중식평균'] = test['요일'].apply(lambda x : lunch_day_mean.loc[x, '중식계'])
test['요일석식평균'] = test['요일'].apply(lambda x : dinner_day_mean.loc[x, '석식계'])
test['월중식평균'] = test['월'].apply(lambda x : lunch_mon_mean.loc[x, '중식계'])
test['월석식평균'] = test['월'].apply(lambda x : dinner_mon_mean.loc[x, '석식계'])

train['연말'] = 0
train.loc[(train['월'] == 12) & (train['일']>=21), '연말'] = 1

print('done')

done


In [3]:
# 변수들의 시간적 효과 고려

df = pd.concat([train,test],axis = 0).reset_index(drop = True)
vacation_yesterday = []
vacation_tomarrow = []
vacation_ratio_yesterday = []
vacation_ratio_tomarrow = []
night_yesterday = []
night_tomarrow = []
night_ratio_yesterday = []
night_ratio_tomarrow = []
home_yesterday = []
home_tomarrow = []
home_ratio_yesterday = []
home_ratio_tomarrow = []
for i in range(len(df)):
  if df['요일'][i] == 0:
    vacation_yesterday.append(0)
    vacation_ratio_yesterday.append(0)
    night_yesterday.append(0)
    night_ratio_yesterday.append(0)
    home_yesterday.append(0)
    home_ratio_yesterday.append(0)  
    vacation_tomarrow.append(df['본사휴가자수'][i + 1])
    vacation_ratio_tomarrow.append(df['휴가비율'][i + 1])
    night_tomarrow.append(df['본사시간외근무명령서승인건수'][i + 1])
    night_ratio_tomarrow.append(df['야근비율'][i + 1])
    home_tomarrow.append(df['현본사소속재택근무자수'][i + 1])
    home_ratio_tomarrow.append(df['재택비율'][i + 1])
  elif df['요일'][i] == 4:
    vacation_tomarrow.append(0)
    vacation_ratio_tomarrow.append(0)
    night_tomarrow.append(0)
    night_ratio_tomarrow.append(0)
    home_tomarrow.append(0)
    home_ratio_tomarrow.append(0)  
    vacation_yesterday.append(df['본사휴가자수'][i - 1])
    vacation_ratio_yesterday.append(df['휴가비율'][i - 1])
    night_yesterday.append(df['본사시간외근무명령서승인건수'][i - 1])
    night_ratio_yesterday.append(df['야근비율'][i - 1])
    home_yesterday.append(df['현본사소속재택근무자수'][i - 1])
    home_ratio_yesterday.append(df['재택비율'][i - 1])
  else:
    vacation_tomarrow.append(df['본사휴가자수'][i + 1])
    vacation_ratio_tomarrow.append(df['휴가비율'][i + 1])
    night_tomarrow.append(df['본사시간외근무명령서승인건수'][i + 1])
    night_ratio_tomarrow.append(df['야근비율'][i + 1])
    home_tomarrow.append(df['현본사소속재택근무자수'][i + 1])
    home_ratio_tomarrow.append(df['재택비율'][i + 1])
    vacation_yesterday.append(df['본사휴가자수'][i - 1])
    vacation_ratio_yesterday.append(df['휴가비율'][i - 1])
    night_yesterday.append(df['본사시간외근무명령서승인건수'][i - 1])
    night_ratio_yesterday.append(df['야근비율'][i - 1])
    home_yesterday.append(df['현본사소속재택근무자수'][i - 1])
    home_ratio_yesterday.append(df['재택비율'][i - 1])
df['본사휴가자수-1'] = vacation_yesterday
df['본사휴가자수+1'] = vacation_tomarrow
df['휴가비율-1'] = vacation_ratio_yesterday
df['휴가비율+1'] = vacation_ratio_tomarrow
df['본사시간외근무명령서승인건수-1'] = night_yesterday
df['본사시간외근무명령서승인건수+1'] = night_tomarrow
df['야근비율-1'] = night_ratio_yesterday
df['야근비율+1'] = night_ratio_tomarrow
df['현본사소속재택근무자수-1'] = home_yesterday
df['현본사소속재택근무자수+1'] = home_tomarrow
df['재택비율-1'] = home_ratio_yesterday
df['재택비율+1'] = home_ratio_tomarrow
train = df.iloc[:1205]
test = df.iloc[1205:].reset_index(drop= True).drop(['중식계','석식계'],axis =1)

In [4]:
# 계절 추가
def season_define(x):
  if x in [3,4,5]:
    return 0
  elif x in [6,7,8]:
    return 1
  elif x in [9,10,11]:
    return 2
  else:
    return 3
train['계절'] = train['월'].apply(season_define)
test['계절'] = test['월'].apply(season_define)
seasondf = pd.get_dummies(pd.concat([train[['계절']], test[['계절']]], axis = 0),columns = ['계절']).iloc[:1205,:]
seasondftest = pd.get_dummies(pd.concat([train[['계절']], test[['계절']]], axis = 0),columns = ['계절']).iloc[1205:,:].reset_index().drop(['index'],axis = 1)
seasondftest

,계절_0,계절_1,계절_2,계절_3
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
5,0,0,0,1
6,0,0,0,1
7,0,0,0,1
8,0,0,0,1
9,0,0,0,1


### 메뉴의 경우 매 식사의 세번째 메뉴를 주메뉴로 보고 주메뉴들을 모두 직접 분류하여 표를 만들어 사용하였습니다.

In [5]:
!pip install openpyxl

In [6]:
# 메뉴
!pip install xlrd
menu = pd.read_excel('데이콘/주메뉴.xlsx', engine = 'openpyxl')
menu = menu.drop(['Unnamed: 0'],axis = 1).dropna()
menu['메뉴'] = menu['메뉴'].apply(lambda x : x.split('*')[0] if '*' in x else (x.split('&')[0] if '&' in x else (x.split('/')[0] if '/' in x else x)))
train.replace({'석식메뉴': '카레라이스 (쌀:국내산,돈육:국내 맑은국  생선까스*탈탈소스  멕시칸샐러드 (스모크햄:외국산) 오복지무침  포기김치 (김치:국내산) '}, 
              {'석식메뉴': '카레라이스 (쌀:국내산,돈육:국내) 맑은국  생선까스*탈탈소스  멕시칸샐러드 (스모크햄:외국산) 오복지무침  포기김치 (김치:국내산) '}, inplace = True)
train.replace({'석식메뉴': '카레라이스 (쌀:국내산,돈육:국내 미소시루  감자만두*양념  애기새송이버섯볶음  골뱅이야채무침  포기김치 (김치:국내산) '}, 
              {'석식메뉴': '카레라이스 (쌀:국내산,돈육:국내) 미소시루  감자만두*양념  애기새송이버섯볶음  골뱅이야채무침  포기김치 (김치:국내산) '}, inplace = True)

In [7]:
import re
train2 = train.copy()
train1 = train.copy()
test2 = test.copy()
test1 = test.copy()
r = '\([^)]*\)'
train2['중식메뉴_split'] = train['중식메뉴'].apply(lambda x : (re.sub(r, ' ', x).replace('/',' ').replace('*', ' ').replace('&', ' ').replace('(new)', '').split()))
train2['석식메뉴_split'] = train['석식메뉴'].apply(lambda x : (re.sub(r, ' ', x).replace('/',' ').replace('*', ' ').replace('&', ' ').replace('(new)', '').split()))
test2['중식메뉴_split'] = test['중식메뉴'].apply(lambda x : (re.sub(r, ' ', x).replace('/',' ').replace('*', ' ').replace('&', ' ').replace('(new)', '').split()))
test2['석식메뉴_split'] = test['석식메뉴'].apply(lambda x : (re.sub(r, ' ', x).replace('/',' ').replace('*', ' ').replace('&', ' ').replace('(new)', '').split()))

def get_food_embedding(x):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    return x_
train1['중식메뉴_split'] = train['중식메뉴'].apply(lambda x: get_food_embedding(x))
train1['석식메뉴_split'] = train['석식메뉴'].apply(lambda x: get_food_embedding(x))
test1['중식메뉴_split'] = test['중식메뉴'].apply(lambda x: get_food_embedding(x))
test1['석식메뉴_split'] = test['석식메뉴'].apply(lambda x: get_food_embedding(x))

In [8]:
#train1 메뉴 처리 

lunch_lst = []
lunch_count = []
pref = 0
for i in tqdm(range(1205)):
    try:
        if train1['중식계'][i] >= 880:
              pref = 1
        else:
              pref = 0

        lunch_lst.append(train1['중식메뉴_split'][i])
        lunch_count.append(pref)
    except:
        pass
# for i in range(len(lunch_lst)):
  # lunch_lst[i] = lunch_lst[i].replace(',', ' ')
print('done')

dinner_lst = []
dinner_count = []
pref_d = 0
for i in tqdm(range(1205)):
    try:
        if train1['석식계'][i] > 476:
            pref_d = 1
        else:
            pref_d = 0

        dinner_lst.append(train1['석식메뉴_split'][i])
        dinner_count.append(pref_d)
    except:
        pass
# for i in range(len(dinner_lst)):
#   dinner_lst[i] = dinner_lst[i].replace(',', ' ')
print('done')

lunch_lst_test = []
lunch_count_test = []
for i in tqdm(range(50)):
    try:
        lunch_lst_test.append(test1['중식메뉴_split'][i])
        lunch_count_test.append(0)
    except:
        pass
# for i in range(len(lunch_lst_test)):
#   lunch_lst_test[i] = lunch_lst_test[i].replace(',', ' ')
print('done')

dinner_lst_test = []
dinner_count_test = []
for i in tqdm(range(50)):
    try:
        dinner_lst_test.append(test1['석식메뉴_split'][i])
        dinner_count_test.append(0)
    except:
        pass
# for i in range(len(dinner_lst_test)):
#   dinner_lst_test[i] = dinner_lst_test[i].replace(',', ' ')
print('done')

lunch_df = pd.DataFrame({'중식메뉴':lunch_lst})
dinner_df = pd.DataFrame({'석식메뉴':dinner_lst})
lunch_df_t = pd.DataFrame({'중식메뉴':lunch_lst_test})
dinner_df_t = pd.DataFrame({'석식메뉴':dinner_lst_test})

food = list(menu['메뉴'])
result_lunch = []
result_dinner = []
for k in range(len(lunch_df)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(lunch_df.iloc[k][0])):
        if lunch_df.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == lunch_df.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
    #trigger = trigger / trigger.sum()
    result_lunch.append(trigger)

for k in range(len(dinner_df)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(dinner_df.iloc[k][0])):
        if dinner_df.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == dinner_df.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
    #trigger = trigger / trigger.sum()
    result_dinner.append(trigger)
    
result_lunch_t = []
result_dinner_t = []
for k in range(len(lunch_df_t)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(lunch_df_t.iloc[k][0])):
        if lunch_df_t.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == lunch_df_t.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
    # trigger = trigger / trigger.sum()
    result_lunch_t.append(trigger)

for k in range(len(dinner_df_t)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(dinner_df_t.iloc[k][0])):
        if dinner_df_t.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == dinner_df_t.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
    #trigger = trigger / trigger.sum()
    result_dinner_t.append(trigger)
    
column = list(menu.columns[1:-1])
lunch_sort1 = pd.DataFrame(result_lunch ,columns=column)
dinner_sort1 = pd.DataFrame(result_dinner, columns = column)
lunch_sort_t1 = pd.DataFrame(result_lunch_t, columns=column)
dinner_sort_t1 = pd.DataFrame(result_dinner_t, columns = column)

  0%|          | 0/1205 [00:00<?, ?it/s]

done


  0%|          | 0/1205 [00:00<?, ?it/s]

done


  0%|          | 0/50 [00:00<?, ?it/s]

done


  0%|          | 0/50 [00:00<?, ?it/s]

done


In [9]:
lunch_sort1

,곡류,두류,난류,묵류,어패류,육류,채소류,해조류,떡류,양념.및.장류,김치류,만두류,면류,과일류,밥류,죽류,덮밥국밥류,비빔밥볶음밥류,김.초.밥류,국수류,국탕류,찌개류,구이류,무침류,볶음류,장아찌류,전류,조림류,찜류,튀김류,샐러드류,단품류,유제품,빵과자류
0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1201,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1202,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1203,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [10]:
# train2 메뉴 처리

lunch_lst = []
lunch_count = []
pref = 0
for i in tqdm(range(1205)):
    try:
        if train2['중식계'][i] >= 880:
              pref = 1
        else:
              pref = 0

        lunch_lst.append(train2['중식메뉴_split'][i])
        lunch_count.append(pref)
    except:
        pass
# for i in range(len(lunch_lst)):
  # lunch_lst[i] = lunch_lst[i].replace(',', ' ')
print('done')

dinner_lst = []
dinner_count = []
pref_d = 0
for i in tqdm(range(1205)):
    try:
        if train2['석식계'][i] > 476:
            pref_d = 1
        else:
            pref_d = 0

        dinner_lst.append(train2['석식메뉴_split'][i])
        dinner_count.append(pref_d)
    except:
        pass
# for i in range(len(dinner_lst)):
#   dinner_lst[i] = dinner_lst[i].replace(',', ' ')
print('done')

lunch_lst_test = []
lunch_count_test = []
for i in tqdm(range(50)):
    try:
        lunch_lst_test.append(test1['중식메뉴_split'][i])
        lunch_count_test.append(0)
    except:
        pass
# for i in range(len(lunch_lst_test)):
#   lunch_lst_test[i] = lunch_lst_test[i].replace(',', ' ')
print('done')

dinner_lst_test = []
dinner_count_test = []
for i in tqdm(range(50)):
    try:
        dinner_lst_test.append(test1['석식메뉴_split'][i])
        dinner_count_test.append(0)
    except:
        pass
# for i in range(len(dinner_lst_test)):
#   dinner_lst_test[i] = dinner_lst_test[i].replace(',', ' ')
print('done')

lunch_df = pd.DataFrame({'중식메뉴':lunch_lst})
dinner_df = pd.DataFrame({'석식메뉴':dinner_lst})
lunch_df_t = pd.DataFrame({'중식메뉴':lunch_lst_test})
dinner_df_t = pd.DataFrame({'석식메뉴':dinner_lst_test})

food = list(menu['메뉴'])
result_lunch = []
result_dinner = []
for k in range(len(lunch_df)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(lunch_df.iloc[k][0])):
        if lunch_df.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == lunch_df.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
    #trigger = trigger / trigger.sum()
    result_lunch.append(trigger)

for k in range(len(dinner_df)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(dinner_df.iloc[k][0])):
        if dinner_df.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == dinner_df.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
    #trigger = trigger / trigger.sum()
    result_dinner.append(trigger)
    
result_lunch_t = []
result_dinner_t = []
for k in range(len(lunch_df_t)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(lunch_df_t.iloc[k][0])):
        if lunch_df_t.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == lunch_df_t.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
    # trigger = trigger / trigger.sum()
    result_lunch_t.append(trigger)

for k in range(len(dinner_df_t)):
    trigger = np.zeros(34, dtype='int')
    for i in range(len(dinner_df_t.iloc[k][0])):
        if dinner_df_t.iloc[k][0][i] in food:
            trigger += np.array(menu[menu['메뉴'] == dinner_df_t.iloc[k][0][i]].iloc[0][1:-1], dtype='int')
    #trigger = trigger / trigger.sum()
    result_dinner_t.append(trigger)
    
column = list(menu.columns[1:-1])
lunch_sort2 = pd.DataFrame(result_lunch ,columns=column)
dinner_sort2 = pd.DataFrame(result_dinner, columns = column)
lunch_sort_t2 = pd.DataFrame(result_lunch_t, columns=column)
dinner_sort_t2 = pd.DataFrame(result_dinner_t, columns = column)

  0%|          | 0/1205 [00:00<?, ?it/s]

done


  0%|          | 0/1205 [00:00<?, ?it/s]

done


  0%|          | 0/50 [00:00<?, ?it/s]

done


  0%|          | 0/50 [00:00<?, ?it/s]

done


In [11]:
# train1  공휴일 변수 추가


train1['공휴일전후'] = 0
test1['공휴일전후'] = 0

train1['공휴일전후'][17] = 1
train1['공휴일전후'][3] = 1
train1['공휴일전후'][62] = 1
# train1['공휴일전후'][67] = 1
# train1['공휴일전후'][82] = 1
train1['공휴일전후'][131] = 1
# train1['공휴일전후'][130] = 1
train1['공휴일전후'][152] = 1
train1['공휴일전후'][226] = 1
train1['공휴일전후'][221] = 1
train1['공휴일전후'][224] = 1
# train1['공휴일전후'][244] = 1
train1['공휴일전후'][245] = 1
# train1['공휴일전후'][267] = 1
train1['공휴일전후'][310] = 2
train1['공휴일전후'][311] = 1
train1['공휴일전후'][309] = 1
train1['공휴일전후'][330] = 1
train1['공휴일전후'][379] = 1
train1['공휴일전후'][467] = 1
# train1['공휴일전후'][469] = 1
train1['공휴일전후'][470] = 1
train1['공휴일전후'][502] = 2
# train1['공휴일전후'][501] = 1
# train1['공휴일전후'][511] = 1
train1['공휴일전후'][565] = 1
train1['공휴일전후'][623] = 1
train1['공휴일전후'][651] = 1
# train1['공휴일전후'][650] = 1
train1['공휴일전후'][705] = 1
# train1['공휴일전후'][707] = 1
train1['공휴일전후'][709] = 1
# train1['공휴일전후'][733] = 1
# train1['공휴일전후'][748] = 1
# train1['공휴일전후'][792] = 1
train1['공휴일전후'][815] = 1
train1['공휴일전후'][864] = 1
# train1['공휴일전후'][863] = 1
train1['공휴일전후'][950] = 1
train1['공휴일전후'][951] = 1
train1['공휴일전후'][953] = 1
train1['공휴일전후'][954] = 1
train1['공휴일전후'][955] = 1
train1['공휴일전후'][971] = 2
# train1['공휴일전후'][970] = 1
# train1['공휴일전후'][1037] = 1
train1['공휴일전후'][1038] = 1
train1['공휴일전후'][1099] = 1
train1['공휴일전후'][1129] = 1
# train1['공휴일전후'][1128] = 1
train1['공휴일전후'][1187] = 1
# train1['공휴일전후'][1186] = 1

test1['공휴일전후'][10] =2
test1['공휴일전후'][20] = 1

train1 = pd.get_dummies(train1, columns=['공휴일전후'])
test1 = pd.get_dummies(test1, columns=['공휴일전후'])

test1['공휴일전후_0'][20] =1
test1['공휴일전후_1'][20] = 0

In [12]:
# train2  공휴일 변수 추가


train2['공휴일전후'] = 0
test2['공휴일전후'] = 0

train2['공휴일전후'][17] = 1
train2['공휴일전후'][3] = 1
train2['공휴일전후'][62] = 1
# train2['공휴일전후'][67] = 1
# train2['공휴일전후'][82] = 1
train2['공휴일전후'][131] = 1
# train2['공휴일전후'][130] = 1
train2['공휴일전후'][152] = 1
train2['공휴일전후'][226] = 1
train2['공휴일전후'][221] = 1
train2['공휴일전후'][224] = 1
# train2['공휴일전후'][244] = 1
train2['공휴일전후'][245] = 1
# train2['공휴일전후'][267] = 1
train2['공휴일전후'][310] = 2
train2['공휴일전후'][311] = 1
train2['공휴일전후'][309] = 1
train2['공휴일전후'][330] = 1
train2['공휴일전후'][379] = 1
train2['공휴일전후'][467] = 1
# train2['공휴일전후'][469] = 1
train2['공휴일전후'][470] = 1
train2['공휴일전후'][502] = 2
# train2['공휴일전후'][501] = 1
# train2['공휴일전후'][511] = 1
train2['공휴일전후'][565] = 1
train2['공휴일전후'][623] = 1
train2['공휴일전후'][651] = 1
# train2['공휴일전후'][650] = 1
train2['공휴일전후'][705] = 1
# train2['공휴일전후'][707] = 1
train2['공휴일전후'][709] = 1
# train2['공휴일전후'][733] = 1
# train2['공휴일전후'][748] = 1
# train2['공휴일전후'][792] = 1
train2['공휴일전후'][815] = 1
train2['공휴일전후'][864] = 1
# train2['공휴일전후'][863] = 1
train2['공휴일전후'][950] = 1
train2['공휴일전후'][951] = 1
train2['공휴일전후'][953] = 1
train2['공휴일전후'][954] = 1
train2['공휴일전후'][955] = 1
train2['공휴일전후'][971] = 2
# train2['공휴일전후'][970] = 1
# train2['공휴일전후'][1037] = 1
train2['공휴일전후'][1038] = 1
train2['공휴일전후'][1099] = 1
train2['공휴일전후'][1129] = 1
# train2['공휴일전후'][1128] = 1
train2['공휴일전후'][1187] = 1
# train2['공휴일전후'][1186] = 1

test2['공휴일전후'][10] =2
test2['공휴일전후'][20] = 1

train2 = pd.get_dummies(train2, columns=['공휴일전후'])
test2 = pd.get_dummies(test2, columns=['공휴일전후'])

test2['공휴일전후_0'][20] =1
test2['공휴일전후_1'][20] = 0

# 외부 변수

In [13]:
#코로나 확진자, 코로나 검색량, 날씨, 코스피지수, 소비자물가지수, 비트코인가격 추가

covid = pd.read_excel('데이콘/코로나감염자.xlsx', engine = 'openpyxl')
covidtrend = pd.read_excel('데이콘/datalab (1).xlsx', engine = 'openpyxl')
weather = pd.read_csv('기상진주.csv', encoding = 'cp949')
weathertest = pd.read_csv('기상진주test.csv', encoding = 'cp949')
kospi = pd.read_csv('코스피코스닥.csv', encoding= 'cp949')
bitcoin = pd.read_excel('데이콘/비트코인(네이버).xlsx', engine = 'openpyxl')
customprice = pd.read_excel('데이콘/소비자물가.xlsx', engine = 'openpyxl').dropna()
covid['날짜'] = covid['날짜'].astype('string')
covidtrend['날짜'] = covidtrend['날짜'].astype('string')

train1['일자'] = train1['일자'].astype('string')
test1['일자'] = test1['일자'].astype('string')

train2['일자'] = train2['일자'].astype('string')
test2['일자'] = test2['일자'].astype('string')

In [14]:
#train1
customprice['년'] = customprice['날짜'].apply(lambda x : int(x[:4]))
customprice['월'] = customprice['날짜'].apply(lambda x : int(x[5:]))
train1 = pd.merge(train1, customprice,how = 'left').drop(['날짜','소비자물가지수-1'], axis = 1)
test1 = pd.merge(test1, customprice,how = 'left').drop(['날짜','소비자물가지수-1'], axis = 1)

train1 = pd.merge(train1, kospi.fillna(method="ffill"), left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1)
train1 = pd.merge(train1, bitcoin, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1)
train1['KOSPI지수'] = train1['KOSPI지수'].apply(lambda x: float(x.replace(',','')))

test1 = pd.merge(test1, kospi.fillna(method="ffill"), left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1)
test1 = pd.merge(test1, bitcoin, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1)
test1['KOSPI지수'] = test1['KOSPI지수'].apply(lambda x: float(x.replace(',','')))

covid['날짜'] = covid['날짜'].astype('string')
covidtrend['날짜'] = covidtrend['날짜'].astype('string')
train1['일자'] = train1['일자'].astype('string')
test1['일자'] = test1['일자'].astype('string')

train1 = pd.merge(pd.merge(train1, covid, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1), covidtrend, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1).fillna(0)
test1 = pd.merge(pd.merge(test1, covid, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1), covidtrend, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1).fillna(0)

train1 = pd.merge(train1,weather[['일시', '최고기온(°C)', '일강수량(mm)', '평균 상대습도(%)', '합계 일조시간(hr)']].fillna(0), left_on = '일자', right_on = '일시').drop(['일시'],axis = 1)
test1 = pd.merge(test1,weathertest[['일시', '최고기온(°C)', '일강수량(mm)', '평균 상대습도(%)', '합계 일조시간(hr)']].fillna(0), left_on = '일자', right_on = '일시').drop(['일시'],axis = 1)

In [15]:
#train2
customprice['년'] = customprice['날짜'].apply(lambda x : int(x[:4]))
customprice['월'] = customprice['날짜'].apply(lambda x : int(x[5:]))
train2 = pd.merge(train2, customprice,how = 'left').drop(['날짜','소비자물가지수-1'], axis = 1)
test2 = pd.merge(test2, customprice,how = 'left').drop(['날짜','소비자물가지수-1'], axis = 1)

train2 = pd.merge(train2, kospi.fillna(method="ffill"), left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1)
train2 = pd.merge(train2, bitcoin, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1)
train2['KOSPI지수'] = train2['KOSPI지수'].apply(lambda x: float(x.replace(',','')))

test2 = pd.merge(test2, kospi.fillna(method="ffill"), left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1)
test2 = pd.merge(test2, bitcoin, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1)
test2['KOSPI지수'] = test2['KOSPI지수'].apply(lambda x: float(x.replace(',','')))

covid['날짜'] = covid['날짜'].astype('string')
covidtrend['날짜'] = covidtrend['날짜'].astype('string')
train2['일자'] = train2['일자'].astype('string')
test2['일자'] = test2['일자'].astype('string')

train2 = pd.merge(pd.merge(train2, covid, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1), covidtrend, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1).fillna(0)
test2 = pd.merge(pd.merge(test2, covid, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1), covidtrend, left_on = '일자', right_on = '날짜', how = 'left').drop(['날짜'], axis = 1).fillna(0)

train2 = pd.merge(train2,weather[['일시', '최고기온(°C)', '일강수량(mm)', '평균 상대습도(%)', '합계 일조시간(hr)']].fillna(0), left_on = '일자', right_on = '일시').drop(['일시'],axis = 1)
test2 = pd.merge(test2,weathertest[['일시', '최고기온(°C)', '일강수량(mm)', '평균 상대습도(%)', '합계 일조시간(hr)']].fillna(0), left_on = '일자', right_on = '일시').drop(['일시'],axis = 1)

# 최종 변수

In [16]:
# 점심

xcolumns_lunch = [
  '본사정원수',
 '본사휴가자수',
 '본사출장자수',
 '본사시간외근무명령서승인건수',
 '현본사소속재택근무자수',
 #'최고기온(°C)',
 #'일강수량(mm)',
 '출근',
 '휴가비율',
 '출장비율',
 '야근비율',
 '재택비율',
 '공휴일전후_0',
 '공휴일전후_1',
 '공휴일전후_2',
 #'감염자',
 '감염자-1',
# '사망자',
 '사망자-1',
# '검색량',
 '검색량-1',
 '년',
 '월',
 '일',
 '주',
 '요일', 
 #'평균 상대습도(%)', '합계 일조시간(hr)', '소비자물가지수전월대비', 'KOSPI지수', 'KOSDAQ지수'
 ]

In [17]:
# 저녁

xcolumns_dinner = [
  '본사정원수',
'본사휴가자수',
 '본사출장자수',
 '본사시간외근무명령서승인건수',
 '현본사소속재택근무자수',
#'최고기온(°C)',
#'일강수량(mm)',
 '출근',
 '휴가비율',
 '출장비율',
 '야근비율',
 '재택비율',
 '메뉴x', 
 #'요일중식평균',
 '요일석식평균', 
 #'월중식평균', 
 '월석식평균', 
 '연말',
 '식사가능자수', 
 '본사휴가자수-1', '본사휴가자수+1',
 '본사시간외근무명령서승인건수-1', '본사시간외근무명령서승인건수+1',
 '현본사소속재택근무자수-1', 
 '현본사소속재택근무자수+1', 
 '공휴일전후_0',
 '공휴일전후_1',
 '공휴일전후_2',
 '감염자',
 '감염자-1',
 #'사망자',
 '사망자-1',
 #'검색량',
 '검색량-1',
 '년',
 '월',
 '일',
 '주',
 '요일', 
 #'평균 상대습도(%)', '합계 일조시간(hr)', 
 #'소비자물가지수전월대비', 'KOSPI지수', 
 #'KOSDAQ지수'
 #'비트코인', '재난지원',
  #'소비자물가지수',
 ]

In [18]:
train_lunch = train1[xcolumns_lunch]
test_lunch = test1[xcolumns_lunch]
train_dinner = train2[xcolumns_dinner]
test_dinner = test2[xcolumns_dinner]

In [19]:
train_lunch = pd.concat([train_lunch, lunch_sort1, seasondf], axis = 1)
train_dinner = pd.concat([train_dinner, dinner_sort2,seasondf], axis = 1)
test_lunch = pd.concat([test_lunch, lunch_sort_t1,seasondftest], axis = 1)
test_dinner = pd.concat([test_dinner, dinner_sort_t2,seasondftest], axis = 1)

# 분석

In [20]:
tmpt = pd.read_csv('데이콘/train.csv')

## 점심

In [21]:
reg = setup(data=pd.concat([train_lunch, tmpt['중식계']], axis = 1),
            target='중식계',
            numeric_imputation = 'mean',
            normalize = True,
            silent= True,  
            #feature_selection_threshold = 0.7,
            session_id = 2)

,Description,Value
0,session_id,2
1,Target,중식계
2,Original Data,"(1205, 60)"
3,Missing Values,False
4,Numeric Features,22
5,Categorical Features,37
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(843, 99)"


In [22]:
train_lunch.columns# 최종 컬럼!! -> 이 노트북의 최종컬럼

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '출근',
       '휴가비율', '출장비율', '야근비율', '재택비율', '공휴일전후_0', '공휴일전후_1', '공휴일전후_2',
       '감염자-1', '사망자-1', '검색량-1', '년', '월', '일', '주', '요일', '곡류', '두류', '난류',
       '묵류', '어패류', '육류', '채소류', '해조류', '떡류', '양념.및.장류', '김치류', '만두류', '면류',
       '과일류', '밥류', '죽류', '덮밥국밥류', '비빔밥볶음밥류', '김.초.밥류', '국수류', '국탕류', '찌개류',
       '구이류', '무침류', '볶음류', '장아찌류', '전류', '조림류', '찜류', '튀김류', '샐러드류', '단품류',
       '유제품', '빵과자류', '계절_0', '계절_1', '계절_2', '계절_3'],
      dtype='object')

In [23]:
train_dinner.columns# 최종 컬럼!! -> 이 노트북의 최종컬럼

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '출근',
       '휴가비율', '출장비율', '야근비율', '재택비율', '메뉴x', '요일석식평균', '월석식평균', '연말',
       '식사가능자수', '본사휴가자수-1', '본사휴가자수+1', '본사시간외근무명령서승인건수-1',
       '본사시간외근무명령서승인건수+1', '현본사소속재택근무자수-1', '현본사소속재택근무자수+1', '공휴일전후_0',
       '공휴일전후_1', '공휴일전후_2', '감염자', '감염자-1', '사망자-1', '검색량-1', '년', '월', '일',
       '주', '요일', '곡류', '두류', '난류', '묵류', '어패류', '육류', '채소류', '해조류', '떡류',
       '양념.및.장류', '김치류', '만두류', '면류', '과일류', '밥류', '죽류', '덮밥국밥류', '비빔밥볶음밥류',
       '김.초.밥류', '국수류', '국탕류', '찌개류', '구이류', '무침류', '볶음류', '장아찌류', '전류', '조림류',
       '찜류', '튀김류', '샐러드류', '단품류', '유제품', '빵과자류', '계절_0', '계절_1', '계절_2',
       '계절_3'],
      dtype='object')

In [24]:
best_10_l = compare_models(sort = 'MAE', n_select = 10)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,66.8602,8162.7243,90.0544,0.8119,0.1125,0.0815,1.0770
gbr,Gradient Boosting Regressor,70.1371,8670.3800,92.7071,0.8002,0.1152,0.0856,0.0390
br,Bayesian Ridge,72.0612,8800.8893,93.6419,0.7975,0.1155,0.0873,0.0070
ridge,Ridge Regression,72.2592,8839.9056,93.8506,0.7963,0.1150,0.0872,0.0090
lasso,Lasso Regression,72.6039,8997.5117,94.5833,0.7931,0.1171,0.0882,0.1730
huber,Huber Regressor,73.2901,9212.3497,95.7413,0.7882,0.1197,0.0891,0.0150
lightgbm,Light Gradient Boosting Machine,73.9859,9886.5227,99.1403,0.7725,0.1237,0.0899,0.0210
rf,Random Forest Regressor,74.2230,9884.4104,99.1329,0.7725,0.1250,0.0911,0.1670
et,Extra Trees Regressor,78.9468,11435.8027,106.5203,0.7375,0.1364,0.0978,0.1610
omp,Orthogonal Matching Pursuit,79.3227,10825.0193,103.7332,0.7512,0.1289,0.0965,0.0060


In [25]:
#상위 8개 모델 앙상블

blended_l = blend_models(estimator_list= best_10_l[:8], fold=5, optimize='MAE')
pred_holdout = predict_model(blended_l)
final_model_l = finalize_model(blended_l)
pred_esb_l = predict_model(final_model_l, test_lunch)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,67.8583,7858.4242,88.6478,0.8265,0.1104,0.0846
1,69.4017,8210.1479,90.6099,0.8042,0.1094,0.0834
2,65.2148,7302.3453,85.4538,0.8347,0.1024,0.0757
3,71.5567,9398.0111,96.9433,0.7828,0.1235,0.0866
4,64.6801,7555.3126,86.9213,0.8359,0.1140,0.0821
Mean,67.7423,8064.8482,89.7152,0.8168,0.1119,0.0825
SD,2.5722,732.4047,4.0038,0.0204,0.0069,0.0037


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,59.7090,6447.7899,80.2981,0.8494,0.0979,0.0722


In [26]:
pred_esb_l['Label']

0      986.540241
1      955.425503
2      641.429733
3     1299.713158
4     1044.585312
5     1006.679380
6     1005.279033
7      761.021010
8     1325.447650
9     1062.845126
10     809.071352
11    1342.303317
12    1149.588622
13    1083.455638
14     987.545231
15     745.495770
16    1310.937592
17    1108.355396
18     942.163348
19     934.827353
20     660.959439
21    1123.448438
22    1025.445012
23     963.963118
24     729.447236
25    1339.479041
26    1155.384562
27    1060.850143
28     986.199544
29     745.463883
30    1357.200840
31    1074.151039
32    1035.322248
33     940.061029
34     701.410510
35    1294.144252
36    1058.125132
37     918.377823
38     870.476647
39     634.760162
40    1272.992551
41    1047.107286
42     982.710529
43     867.470180
44     673.074616
45    1293.775229
46    1091.185554
47     986.799572
48     957.212467
49     703.420249
Name: Label, dtype: float64

In [27]:
submission_esb = pd.read_csv('데이콘/sample_submission.csv')
submission_esb['중식계'] = pred_esb_l['Label']

## 저녁

In [28]:
reg = setup(data=pd.concat([train_dinner, tmpt['석식계']], axis = 1),
            target='석식계',
            numeric_imputation = 'mean',
            normalize = True,
            silent= True, 
            #feature_selection_threshold = 0.7,
            session_id = 6)

,Description,Value
0,session_id,6
1,Target,석식계
2,Original Data,"(1205, 72)"
3,Missing Values,False
4,Numeric Features,32
5,Categorical Features,39
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(843, 111)"


In [29]:
# 상위 10개 모델 앙상블

best_10_d = compare_models(sort = 'MAE', n_select = 10)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,46.3020,4269.0349,64.6970,0.7917,0.7357,0.1061,1.7590
et,Extra Trees Regressor,46.5947,4600.6684,66.7883,0.7747,0.4030,0.1092,0.1680
lightgbm,Light Gradient Boosting Machine,47.3124,4286.6734,64.4406,0.7918,0.6413,0.1120,0.0320
ridge,Ridge Regression,47.3422,4433.2886,65.7479,0.7807,0.7711,0.1095,0.0060
br,Bayesian Ridge,47.3668,4421.4836,65.6457,0.7813,0.7897,0.1089,0.0090
rf,Random Forest Regressor,47.7397,4593.8333,66.8867,0.7762,0.4402,0.1138,0.1890
lasso,Lasso Regression,47.8929,4520.9173,66.2897,0.7768,0.7909,0.1116,0.0080
gbr,Gradient Boosting Regressor,47.9005,4598.1154,66.9477,0.7762,0.7115,0.1115,0.0520
lr,Linear Regression,48.0624,4643.1536,67.3671,0.7702,0.7547,0.1140,0.0090
omp,Orthogonal Matching Pursuit,52.3532,5359.4721,72.5448,0.7347,0.7570,0.1252,0.0080


In [30]:
blended_d = blend_models(estimator_list = best_10_d[:5], fold = 5, optimize = 'MAE')
pred_holdout = predict_model(blended_d)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,41.4795,3352.4528,57.9004,0.8282,0.9185,0.0855
1,51.0335,5428.8205,73.6805,0.7512,0.6195,0.1300
2,40.3088,2740.2324,52.3472,0.8345,0.5641,0.0918
3,45.0490,4872.1897,69.8011,0.7940,0.8477,0.1033
4,44.1929,3191.2062,56.4908,0.8505,0.5594,0.1052
Mean,44.4127,3916.9803,62.0440,0.8117,0.7018,0.1032
SD,3.7345,1041.9463,8.2173,0.0354,0.1512,0.0153


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,39.3508,3011.9886,54.8816,0.8141,0.4462,0.0926


In [31]:
final_model_d = finalize_model(blended_d)
pred_esb_d = predict_model(final_model_d, test_dinner)

In [32]:
pred_esb_d['Label']

0     377.870660
1     410.203399
2     239.205640
3     561.710865
4     478.695673
5     423.732474
6     485.434050
7     347.446235
8     626.738286
9     514.218395
10    163.770605
11    688.162125
12    637.512835
13    466.306994
14    527.092797
15    344.324834
16    664.169760
17    602.370226
18    408.159260
19    503.804626
20    297.831649
21    623.002788
22    472.711679
23    545.227150
24    368.116467
25    698.340742
26    671.625150
27    462.491496
28    543.802481
29    332.619633
30    695.862578
31    600.371991
32    437.430569
33    491.942011
34    300.623097
35    641.509869
36    590.959660
37    397.259097
38    459.908941
39    284.962124
40    637.081193
41    592.602565
42    419.270752
43    440.579417
44    252.794831
45    604.288650
46    554.675165
47    435.322482
48    457.724629
49    289.264271
Name: Label, dtype: float64

## 최종 결과

In [33]:
submission_esb['석식계'] = pred_esb_d['Label']
submission_esb

,일자,중식계,석식계
0,2021-01-27,986.540241,377.870660
1,2021-01-28,955.425503,410.203399
2,2021-01-29,641.429733,239.205640
3,2021-02-01,1299.713158,561.710865
4,2021-02-02,1044.585312,478.695673
5,2021-02-03,1006.679380,423.732474
6,2021-02-04,1005.279033,485.434050
7,2021-02-05,761.021010,347.446235
8,2021-02-08,1325.447650,626.738286
9,2021-02-09,1062.845126,514.218395


In [34]:
submission_esb.to_csv('submission.csv', index = False)